In [1]:
from numpy import *
import numpy as np

In [2]:
U , Sigma , VT = np.linalg.svd([[1,1],[7,7]])

In [3]:
U

array([[-0.14142136, -0.98994949],
       [-0.98994949,  0.14142136]])

In [4]:
Sigma

array([10.,  0.])

In [5]:
VT

array([[-0.70710678, -0.70710678],
       [-0.70710678,  0.70710678]])

In [6]:
def loadExData():
    return[[0, 0, 0, 2, 2],
           [0, 0, 0, 3, 3],
           [0, 0, 0, 1, 1],
           [1, 1, 1, 0, 0],
           [2, 2, 2, 0, 0],
           [5, 5, 5, 0, 0],
           [1, 1, 1, 0, 0]]

In [7]:
def loadExData2():
    return[[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 5],
           [0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 3],
           [0, 0, 0, 0, 4, 0, 0, 1, 0, 4, 0],
           [3, 3, 4, 0, 0, 0, 0, 2, 2, 0, 0],
           [5, 4, 5, 0, 0, 0, 0, 5, 5, 0, 0],
           [0, 0, 0, 0, 5, 0, 1, 0, 0, 5, 0],
           [4, 3, 4, 0, 0, 0, 0, 5, 5, 0, 1],
           [0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 4],
           [0, 0, 0, 2, 0, 2, 5, 0, 0, 1, 2],
           [0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 0],
           [1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0]]

In [8]:
Data = loadExData()

In [9]:
U, Sigma , VT =np.linalg.svd(Data)

In [10]:
Sigma

array([9.64365076e+00, 5.29150262e+00, 8.36478329e-16, 6.91811207e-17,
       3.76946717e-34])

In [11]:
Sigma3 = np.mat([[Sigma[0] ,0 ,0],[0,Sigma[1],0],[0,0,Sigma[2]]])

In [12]:
U[:,:3] * Sigma3*VT[:3,:]

matrix([[-1.19154932e-15,  8.50091935e-16,  3.41457382e-16,
          2.00000000e+00,  2.00000000e+00],
        [-7.56974048e-16,  2.29691224e-16,  5.27282824e-16,
          3.00000000e+00,  3.00000000e+00],
        [-2.27747782e-16,  5.16267387e-17,  1.76121044e-16,
          1.00000000e+00,  1.00000000e+00],
        [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
          6.97716937e-33,  6.97716937e-33],
        [ 2.00000000e+00,  2.00000000e+00,  2.00000000e+00,
          1.39543387e-32,  1.39543387e-32],
        [ 5.00000000e+00,  5.00000000e+00,  5.00000000e+00,
          2.07703709e-16,  2.07703709e-16],
        [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
          6.97716937e-33,  6.97716937e-33]])

In [13]:
def ecludSim(inA , inB):
    return 1.0 / (1.0 + np.linalg.norm(inA - inB))

In [14]:
def pearsSim(inA,inB):
    if len(inA) < 3 : return 1.0
    return 0.5+0.5*corrcoef(inA, inB, rowvar = 0)[0][1]

In [15]:
def cosSim(inA ,inB):
    num = float(inA.T*inB)
    denom = np.linalg.norm(inA) * np.linalg.norm(inB)
    return 0.5 + 0.5 *(num / denom)

In [16]:
myMat = np.mat(loadExData())

In [17]:
ecludSim(myMat[:,0] ,myMat[:,4])

0.12973190755680383

In [18]:
ecludSim(myMat[:,0],myMat[:,0])

1.0

In [19]:
cosSim(myMat[:,0] , myMat[:,4])

0.5

In [20]:
cosSim(myMat[:,0] , myMat[:,0])

1.0

In [21]:
pearsSim(myMat[:,0],myMat[:,4])

0.20596538173840329

In [22]:
pearsSim(myMat[:,0],myMat[:,0])

1.0

In [23]:
def standEst(dataMat , user,simMeas,item):
    n = shape(dataMat)[1]
    simTotal = 0.0
    ratSimTotal = 0.0
    for j in range(n):
        userRating = dataMat[user ,j]
        if userRating == 0:continue
        overLap = np.nonzero(np.logical_and(dataMat[:,item].A > 0,
                                           dataMat[:,j].A > 0))[0]
        if len(overLap) == 0: similarity =0
        else : similarity = simMeas(dataMat[overLap , item],
                                   dataMat[overLap,j])
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0: return 0
    else : return ratSimTotal / simTotal
    

In [24]:
def recommend(dataMat , user , N=3 ,simMeas=cosSim , estMethod=standEst):
    unratedItems = np.nonzero(dataMat[user,:].A ==0)[1]
    if len(unratedItems) == 0: return ('you rated everthing')
    itemScores = []
    for item in unratedItems :
        estimatedScore = estMethod(dataMat , user ,simMeas ,item)
        itemScores.append((item ,estimatedScore))
    return sorted(itemScores , key=lambda jj: jj[1] , reverse=True)[:N]

In [25]:
myMat  = np.mat(loadExData())

In [26]:
myMat[0,1] = myMat[0,0] = myMat[1,0] = myMat[2,0]  = 4

In [27]:
myMat[3,3] =2

In [28]:
myMat

matrix([[4, 4, 0, 2, 2],
        [4, 0, 0, 3, 3],
        [4, 0, 0, 1, 1],
        [1, 1, 1, 2, 0],
        [2, 2, 2, 0, 0],
        [5, 5, 5, 0, 0],
        [1, 1, 1, 0, 0]])

In [29]:
recommend(myMat ,2)

[(2, 2.5), (1, 2.0243290220056256)]

In [30]:
recommend(myMat ,2 , simMeas=ecludSim)

[(2, 3.0), (1, 2.8266504712098603)]

In [31]:
recommend(myMat , 2,simMeas=pearsSim)

[(2, 2.5), (1, 2.0)]

In [32]:
U , Sigma , VT = np.linalg.svd(loadExData2())

In [33]:
Sigma

array([15.77075346, 11.40670395, 11.03044558,  4.84639758,  3.09292055,
        2.58097379,  1.00413543,  0.72817072,  0.43800353,  0.22082113,
        0.07367823])

In [34]:
Sig2 = Sigma ** 2
np.sum(Sig2)

541.9999999999995

In [35]:
np.sum(Sig2) * 0.9

487.7999999999996

In [36]:
np.sum(Sig2[:2])

378.8295595113579

In [37]:
np.sum(Sig2[:3])

500.5002891275792

In [38]:
def svdEst(dataMat, user, simMeas, item):
    n = shape(dataMat)[1]
    simTotal = 0.0; ratSimTotal = 0.0
    U,Sigma,VT = np.linalg.svd(dataMat)
    Sig4 = mat(eye(4)*Sigma[:4]) #arrange Sig4 into a diagonal matrix
    xformedItems = dataMat.T * U[:,:4] * Sig4.I  #create transformed items
    for j in range(n):
        userRating = dataMat[user,j]
        if userRating == 0 or j==item: continue
        similarity = simMeas(xformedItems[item,:].T,\
                             xformedItems[j,:].T)
        print ('the %d and %d similarity is: %f' % (item, j, similarity))
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0: return 0
    else: return ratSimTotal/simTotal

In [39]:
recommend(myMat , 1 ,estMethod=svdEst)

the 1 and 0 similarity is: 0.498142
the 1 and 3 similarity is: 0.498131
the 1 and 4 similarity is: 0.509974
the 2 and 0 similarity is: 0.552670
the 2 and 3 similarity is: 0.552976
the 2 and 4 similarity is: 0.217301


[(2, 3.4177569186592387), (1, 3.3307171545585645)]

In [40]:
recommend(myMat , 1 ,estMethod=svdEst,simMeas=pearsSim)

the 1 and 0 similarity is: 0.626075
the 1 and 3 similarity is: 0.672793
the 1 and 4 similarity is: 0.614375
the 2 and 0 similarity is: 0.429334
the 2 and 3 similarity is: 0.387057
the 2 and 4 similarity is: 0.043539


[(2, 3.4992661245386785), (1, 3.3272324280613663)]

In [77]:
def printMat(inMat, thresh=0.8):
    outMat = np.mat(np.zeros(shape(inMat)))
    for i in range(32):
        for k in range(32):
            if float(inMat[i,k]) > thresh:
                outMat[i,k] = 1
            else: outMat[i,k] = 0
        print(outMat[i].A.tolist(),'\n')

In [80]:
def imgCompress(numSV=3, thresh=0.8):
    myl = []
    for line in open('0_5.txt').readlines():
        newRow = []
        for i in range(32):
            newRow.append(int(line[i]))
        myl.append(newRow)
    myMat = mat(myl)
    print ("****original matrix******")
    #printMat(myMat, thresh)
    U,Sigma,VT = np.linalg.svd(myMat)
    SigRecon = np.mat(np.zeros((numSV, numSV)))
    for k in range(numSV):
        SigRecon[k,k] = Sigma[k]
    reconMat = U[:,:numSV]*SigRecon*VT[:numSV,:]
    print ("****reconstructed matrix using %d singular values******" % numSV)
    #printMat(reconMat, thresh)
    print(myMat.shape,reconMat.shape)

In [81]:
imgCompress(2)

****original matrix******
****reconstructed matrix using 2 singular values******
(32, 32) (32, 32)
